# Historic followers on Twitter

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [3]:
import altair as alt
import altair_latimes as lat
import numpy as np

In [4]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('latimes')

In [5]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.options.display.max_colwidth = None

### Get json from a user archive

In [7]:
user = "elonmusk"

In [8]:
url = (
    "http://web.archive.org/cdx/search/cdx?url=https://www.twitter.com/"
    + user
    + "&output=json"
)

In [9]:
urls = pd.read_json(url, orient="records").drop([0], axis=0)

In [10]:
urls.columns = [
    "urlkey",
    "timestamp",
    "original",
    "mimetype",
    "statuscode",
    "digest",
    "length",
]

In [11]:
urls["datetime"] = pd.to_datetime(urls["timestamp"])

In [12]:
urls["date"] = urls["datetime"].dt.date

### Limit the update urls to the last one of each day

In [13]:
daily_urls = urls.sort_values(["datetime", "date"], ascending=True).drop_duplicates(
    "date", keep="last"
)

In [14]:
urls_success = daily_urls[daily_urls["statuscode"] == "200"].copy()

In [15]:
len(urls_success)

In [20]:
urls_success["original"] = urls_success["original"].str.replace(
    "http://twitter.com:80", "http://twitter.com", regex=False
)

In [21]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
}

In [55]:
urls_success_recent = urls_success[urls_success["datetime"] > "2022-10-27"].copy()

In [56]:
urls_success_recent["url"] = (
    "https://web.archive.org/web/"
    + urls_success["timestamp"].astype(str)
    + "/"
    + urls_success["original"].astype(str)
)

## Broken

In [66]:
# divs = []

# for t, u in zip(urls_success_recent.timestamp, urls_success_recent.original):
#     r = requests.get("https://web.archive.org/web/" + t + "/" + u, headers=headers)
#     soup = BeautifulSoup(r.text, "html.parser")
#     if soup.find("div", class_="css-1dbjc4n") == None:
#         continue
#     else:
#         divs.append(soup.find("div", class_="css-1dbjc4n"))

In [68]:
# for div in divs:
#     <span class="css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0">110.2M</span>